*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [173]:
# import libraries (you may add additional imports but you may not have to)
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pyplot as plt
import os
import requests
import zipfile

In [179]:
r = requests.get('https://cdn.freecodecamp.org/project-data/books/book-crossings.zip') 
open('book-crossings.zip', 'wb').write(r.content)

with zipfile.ZipFile('book-crossings.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [208]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    # get data files
    r = requests.get('https://cdn.freecodecamp.org/project-data/books/book-crossings.zip') 
    open('book-crossings.zip', 'wb').write(r.content)

    with zipfile.ZipFile('book-crossings.zip', 'r') as zip_ref:
        zip_ref.extractall()
    books_filename = 'BX-Books.csv'
    ratings_filename = 'BX-Book-Ratings.csv'
    users_filename = 'BX-Users.csv'        
            
    # import csv data into dataframes
    df_books = pd.read_csv(
        books_filename,
        encoding = "ISO-8859-1",
        sep=";",
        header=0,
        names=['isbn', 'title', 'author'],
        usecols=['isbn', 'title', 'author'],
        dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

    df_ratings = pd.read_csv(
        ratings_filename,
        encoding = "ISO-8859-1",
        sep=";",
        header=0,
        names=['user', 'isbn', 'rating'],
        usecols=['user', 'isbn', 'rating'],
        dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})   

    df_users = pd.read_csv(
        users_filename,
        encoding = "ISO-8859-1",
        sep=';',
        header=0,
        names=['user', 'location', 'age'],
        usecols=['user', 'location', 'age'],
        dtype={'user': 'int32', 'location': 'str', 'age': 'Int32'})
    
    # Remove users with less than 100 ratings
    df_ratings = df_ratings[df_ratings['user'].map(df_ratings['user'].value_counts()) > 200]

    # Remove books with less than 200 ratings
    df_ratings = df_ratings[df_ratings['isbn'].map(df_ratings['isbn'].value_counts()) > 100]

    # Combining ratings with books on isbn columns
    merge_ratings_books = pd.merge(df_ratings, df_books, on='isbn')
    
    # Rating count per book
    rating_per_bookCount = (merge_ratings_books.groupby(by=['title'])['rating']
                                                        .count()
                                                        .reset_index()
                                                        .rename(columns = {'rating':'ratingCount'})
                                                        [['title', 'ratingCount']]
                                                        )
    
    # Combining total count ratings with ratings
    rating_Counts = merge_ratings_books.merge(rating_per_bookCount, left_on='title', right_on='title', how='left')
    
    # Combining with users
    combined_users_ratings = rating_Counts.merge(df_users, left_on='user', right_on='user', how='left')

    # Convert Dataframe to 2D matrix
    combined_users_ratings_pivot = combined_users_ratings.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
    combined_users_ratings_matrix = csr_matrix(combined_users_ratings_pivot.values)

    # Building the model
    model = NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=15)

    # Training the model
    model.fit(combined_users_ratings_matrix)
    
    distances, indices = model.kneighbors(combined_users_ratings_pivot.loc[book].values.reshape(1, -1), n_neighbors=6)
    
    recommended_books = []
    
    for i in range(1,6):
        recommended_books.append([combined_users_ratings_pivot.index[indices.flatten()[i]], distances.flatten()[i]])
        
    recommended_books = [book, recommended_books]
    
    return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [209]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7230184], ["The Pilot's Wife : A Novel", 0.81926787], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.823043], ['Bel Canto: A Novel', 0.8247875]]]
You haven't passed yet. Keep trying!
